In [1]:
from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.callbacks import TensorBoard
import numpy as np
import pickle
from datetime import datetime

(x_train, _), (x_test, _) = mnist.load_data()

In [2]:
RUN_ON_MNIST = True
L = 4

# Notes:
* Add `Input` argument `channels_first` when using Z3 data and 
  change the input dimensions.
* You will have to manualy add periodic edges for periodic padding before real runn
* **requires** running the transform notebook to get the data into a format for the auto encoder

In [3]:
if RUN_ON_MNIST:
    input_obj = Input(shape=(28, 28, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_obj)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)  

else:
    input_obj = Input(shape=(L*2, L*2, 1))
    x = Conv2D(16, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(input_obj)
    encoded = Conv2D(32, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)
    x = Conv2DTranspose(16, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(encoded)
    decoded = Conv2DTranspose(1, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)

autoencoder = Model(input_obj, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [4]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 8)           0     

In [5]:
if RUN_ON_MNIST:
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format
else:
    with open('pickle_data.pkl', 'rb') as f:
        all_data = pickle.load(f)
    all_data = all_data.astype('float32') / 5.0
    n_records = len(all_data)
    x_train = all_data[: n_records - int(n_records/4)]
    x_test = all_data[int(n_records/4) :]
    x_train = np.reshape(x_train, (len(x_train), L * 2, L * 2, 1))
    x_test = np.reshape(x_test, (len(x_test), L * 2, L * 2, 1))

In [6]:
x_train.shape

(60000, 28, 28, 1)

In [7]:
autoencoder.fit(x_train, x_train,
                epochs=1,
                batch_size=100,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder/{}'.format(datetime.now()))])

600/600 [==============================] - 25s 41ms/step - loss: 0.6956 - val_loss: 0.6927


In [8]:
autoencoder.predict(x_test)

array([[[[0.50005037],
         [0.5000198 ],
         [0.49963325],
         ...,
         [0.4999876 ],
         [0.49989158],
         [0.4992017 ]],

        [[0.49895343],
         [0.49800837],
         [0.49811912],
         ...,
         [0.49937522],
         [0.49937513],
         [0.4989239 ]],

        [[0.4976152 ],
         [0.49698314],
         [0.49759585],
         ...,
         [0.49958554],
         [0.49943295],
         [0.4991196 ]],

        ...,

        [[0.5006103 ],
         [0.503491  ],
         [0.50235   ],
         ...,
         [0.501193  ],
         [0.5007542 ],
         [0.50100416]],

        [[0.5014216 ],
         [0.5016481 ],
         [0.50040656],
         ...,
         [0.50045055],
         [0.50047964],
         [0.5007836 ]],

        [[0.5038245 ],
         [0.49895337],
         [0.49719486],
         ...,
         [0.49775803],
         [0.4979038 ],
         [0.49694884]]],


       [[[0.49148595],
         [0.49523327],
         [0.49